<a href="https://colab.research.google.com/github/MungaiMwangi001/LLMs/blob/main/GPT2FT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load the dataset**

In [3]:
import pandas as pd
from datasets import load_dataset

dataset = load_dataset("mteb/tweet_sentiment_extraction")
df = pd.DataFrame(dataset['train'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/240k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3432 [00:00<?, ? examples/s]

**TOkenizer**

In [4]:
from transformers import GPT2Tokenizer
from datasets import load_dataset


# Loading the dataset to train our model
dataset = load_dataset("mteb/tweet_sentiment_extraction")

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
   return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/tokenizer_config.json
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/tokenizer.json
loading file chat_template.jinja from cache at None


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.55.0",
  "use_cach

Map:   0%|          | 0/26732 [00:00<?, ? examples/s]

Map:   0%|          | 0/3432 [00:00<?, ? examples/s]

**Initialize  the base model**

In [5]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

# Remove the 'id', 'text', and 'label_text' columns as they are not needed for training in this format and cause issues with the data collator
small_train_dataset = small_train_dataset.remove_columns(["id", "text", "label_text"])
small_eval_dataset = small_eval_dataset.remove_columns(["id", "text", "label_text"])

**Initialize the base model**

In [6]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)

# Set the padding token ID in the model's configuration
# Use the tokenizer's pad_token_id
model.config.pad_token_id = tokenizer.pad_token_id

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "ta

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/model.safetensors
All model checkpoint weights were used when initializing GPT2ForSequenceClassification.

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
!pip install evaluate==0.4.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00


**Evaluate Method**

In [8]:
import transformers
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import accuracy_score
import numpy as np
import torch
import logging
import os

# Optional: disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"

# Set up logging
logging.basicConfig(level=logging.INFO)
transformers.logging.set_verbosity_info()

print(f"Transformers version: {transformers.__version__}")
print(f"PyTorch version: {torch.__version__}")

# Calculate steps per epoch
n_gpu = torch.cuda.device_count() or 1
print(f"Using {n_gpu} GPU(s)")

per_device_batch_size = 8  # Increased from 1
grad_accumulation = 1
total_batch_size = per_device_batch_size * n_gpu * grad_accumulation
steps_per_epoch = max(1, len(small_train_dataset) // total_batch_size)
print(f"Steps per epoch: {steps_per_epoch}")

# Compute metrics function using sklearn
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Flatten predictions and labels
    flat_predictions = predictions.flatten()
    flat_labels = np.asarray(labels).flatten()

    # Filter out ignored loss indices (-100)
    valid_indices = flat_labels != -100
    valid_predictions = flat_predictions[valid_indices].astype(int)
    valid_labels = flat_labels[valid_indices].astype(int)

    # Compute accuracy
    return {"accuracy": accuracy_score(valid_labels, valid_predictions)}

training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=per_device_batch_size,
    per_device_eval_batch_size=per_device_batch_size,
    gradient_accumulation_steps=grad_accumulation,
    num_train_epochs=2,
    learning_rate=5e-5,

    # Use the correct parameter name
    eval_strategy="steps",
    eval_steps=steps_per_epoch,

    logging_steps=50,
    save_strategy="epoch",

    remove_unused_columns=False,
    no_cuda=not torch.cuda.is_available(),
    fp16=torch.cuda.is_available(),
    overwrite_output_dir=True,
    report_to="none"
)

# Initialize Data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

# Start training
print("Starting training...")
train_result = trainer.train()
print("Training completed!")

# Save model
trainer.save_model("final_model")
print("Model saved to 'final_model' directory")

# Evaluate
if small_eval_dataset is not None:
    print("Evaluating on validation set...")
    eval_results = trainer.evaluate()
    print(f"Validation results: {eval_results}")


PyTorch: setting up devices
average_tokens_across_devices is True but world size is 1. Setting it to False automatically.


Transformers version: 4.55.0
PyTorch version: 2.6.0+cu124
Using 1 GPU(s)
Steps per epoch: 125


Using auto half precision backend
***** Running training *****
  Num examples = 1,000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 250
  Number of trainable parameters = 124,442,112


Starting training...


Step,Training Loss,Validation Loss,Accuracy
125,1.178200,1.046094,0.464000
250,0.812800,0.870866,0.643000



***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-125
Configuration saved in test_trainer/checkpoint-125/config.json
Model weights saved in test_trainer/checkpoint-125/model.safetensors
Saving Trainer.data_collator.tokenizer by default as Trainer.processing_class is `None`
tokenizer config file saved in test_trainer/checkpoint-125/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-125/special_tokens_map.json

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-250
Configuration saved in test_trainer/checkpoint-250/config.json
Model weights saved in test_trainer/checkpoint-250/model.safetensors
Saving Trainer.data_collator.tokenizer by default as Trainer.processing_class is `None`
tokenizer config file saved in test_trainer/checkpoint-250/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-250/special_

Training completed!


Model weights saved in final_model/model.safetensors
Saving Trainer.data_collator.tokenizer by default as Trainer.processing_class is `None`
tokenizer config file saved in final_model/tokenizer_config.json
Special tokens file saved in final_model/special_tokens_map.json

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


Model saved to 'final_model' directory
Evaluating on validation set...


Validation results: {'eval_loss': 0.8708663582801819, 'eval_accuracy': 0.643, 'eval_runtime': 29.1096, 'eval_samples_per_second': 34.353, 'eval_steps_per_second': 4.294, 'epoch': 2.0}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [9]:
!pip install huggingface_hub

from huggingface_hub import notebook_login
notebook_login()


In [11]:
from huggingface_hub import login
login()  # paste your HF write token when prompted

from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("final_model")
tokenizer = AutoTokenizer.from_pretrained("final_model")

model.push_to_hub("SMungai098/my-finetuned-gpt2")
tokenizer.push_to_hub("SMungai098/my-finetuned-gpt2")


loading configuration file final_model/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2ForSequenceClassification"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "problem_type": "single_label_classification",
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params"

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmp50rpenux/model.safetensors    :   0%|          |  548kB /  498MB            

README.md: 0.00B [00:00, ?B/s]

tokenizer config file saved in /tmp/tmpnwzp1alq/tokenizer_config.json
Special tokens file saved in /tmp/tmpnwzp1alq/special_tokens_map.json
Uploading the following files to SMungai098/my-finetuned-gpt2: vocab.json,tokenizer.json,README.md,merges.txt,tokenizer_config.json,special_tokens_map.json


CommitInfo(commit_url='https://huggingface.co/SMungai098/my-finetuned-gpt2/commit/aea56b8c96d83659795baa711953cb74028af180', commit_message='Upload tokenizer', commit_description='', oid='aea56b8c96d83659795baa711953cb74028af180', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SMungai098/my-finetuned-gpt2', endpoint='https://huggingface.co', repo_type='model', repo_id='SMungai098/my-finetuned-gpt2'), pr_revision=None, pr_num=None)

In [37]:
!pip install torch transformers gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 596.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [7]:
!git config --global user.email "mungaistephen066@gmail.com"
!git config --global user.name "MungaiMwangi001"


In [8]:
import os

# Access the token from environment variable
hf_token = os.getenv("HF_TOKEN")


In [9]:
!rm -rf my-finetuned-gpt2-app  # Clean existing folder

!git clone https://huggingface.co/spaces/SMungai098/my-finetuned-gpt2-app


Cloning into 'my-finetuned-gpt2-app'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (10/10), 3.04 KiB | 1.52 MiB/s, done.


In [10]:
%cd my-finetuned-gpt2-app


/content/my-finetuned-gpt2-app/my-finetuned-gpt2-app


In [12]:
app_code = '''
import gradio as gr
import torch
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer

# Load your fine-tuned model and tokenizer
model_name = "SMungai098/my-finetuned-gpt2"  # replace with your HF repo ID
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2ForSequenceClassification.from_pretrained(model_name)
model.eval()

def predict_sentiment(text):
    # Tokenize input tweet
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()

    # Map class IDs to labels (change if your labels differ)
    label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
    return label_map.get(predicted_class_id, "Unknown")

# Create Gradio interface
demo = gr.Interface(fn=predict_sentiment, inputs="text", outputs="text", title="Twitter Sentiment Analyzer")

demo.launch(share=True)
'''

# Save to app.py
with open("app.py", "w") as f:
    f.write(app_code)


In [13]:
!cat app.py



import gradio as gr
import torch
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer

# Load your fine-tuned model and tokenizer
model_name = "SMungai098/my-finetuned-gpt2"  # replace with your HF repo ID
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2ForSequenceClassification.from_pretrained(model_name)
model.eval()

def predict_sentiment(text):
    # Tokenize input tweet
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()

    # Map class IDs to labels (change if your labels differ)
    label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
    return label_map.get(predicted_class_id, "Unknown")

# Create Gradio interface
demo = gr.Interface(fn=predict_sentiment, inputs="text", outputs="text", title="Twitter Sentiment Analyzer")

demo.launch(share=True)


In [14]:
!git add app.py
!git commit -m "Add greeting app"
!git push


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date
